# Experiments

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# company = "reit"
# company = "yellow_pages"
company = "calyxt"

test_pdf_path = f"../data/eval/{company}.pdf"

## Docling - strong!

In [3]:
# Potential for parallelization for files?
from docling.document_converter import DocumentConverter

# source = "https://arxiv.org/pdf/2408.09869"  # document per local path or URL
converter = DocumentConverter()
result = converter.convert(test_pdf_path)
print(result.document.export_to_markdown())

# REIT: execution for 7.4MB with 97 pages -> 5 min
# Yellow Pages: execution for 1.4MB with 77 pages -> 4 min
# Calyxt: execution for 0.7MB with 88 pages -> 3min

KeyboardInterrupt: 

In [ ]:
# domain knwoledge
# brackets in tables indicate negative values
# parsing issues (see current system prompt)

In [22]:
# STORE as md with correct utf-8 encoding
with open(f"../dev/docling_{company}.md", "w", encoding="utf-8") as file:
    file.write(result.document.export_to_markdown())

## Not working well

### PyMuPDF4LLM - tables bad

In [ ]:
## Observations
# tables are pretty bad
# format in general seems ok

In [6]:
# Store as file
import pathlib
pathlib.Path("../dev/pymupdf4llm_output.md").write_bytes(md_text.encode())

261957

In [5]:
md_text[:1000]

'-----\n\n# Table of Contents\n\nManagement’s Discussion and Analysis........................................................................................................ 2\n\nIndependent Auditor’s Report ................................................................................................................. 29-32\n\nConsolidated Statements of Income and Other Comprehensive Income ..................................................... 33\n\nConsolidated Statements of Financial Position ............................................................................................ 34\n\nConsolidated Statements of Changes in Equity ...................................................................................... 35-36\n\nConsolidated Statements of Cash Flows ..................................................................................................... 37\n\nNotes To The Consolidated Financial Statements .................................................................

### Camelot (table detection) - Not working well

In [ ]:
import camelot

# Extract tables from all pages of the PDF
tables = camelot.read_pdf(test_pdf_path, pages="all")

# Export each table to CSV (or process further as needed)
for i, table in enumerate(tables):
    csv_filename = f"table_{i}.csv"
    table.to_csv(csv_filename)
    print(f"Exported table {i} to {csv_filename}")
